In [1]:
import re
import os
import pandas as pd

In [2]:
iemocap_dir = '/home/user/shaheen.perveen/SER_new/IEMOCAP_text/'
labels_df = pd.read_csv('/home/user/shaheen.perveen/SER_new/preprocess_info/df_iemocap.csv')
save_dir = '/home/user/shaheen.perveen/SER_new/preprocess_info/'

In [42]:
text_df = pd.DataFrame(columns=["filename", "text"])

In [43]:
# useful_regex = re.compile(r'^(\w+)', re.IGNORECASE)

for sess in range(1, 6):
    file_path = '{}Session{}/'.format(iemocap_dir, sess)
    evaluation_files = [l for l in os.listdir(file_path) if '_Ses' not in l]
    print(file_path)
    for file in evaluation_files:
        with open(file_path + file, "rt") as f:
            file_content = f.readlines()
            try:
                for myline in file_content:
                    audio_code, rest = myline.split(" ", 1)
    #                 print(audio_code)
                    text = myline.split(':')[-1].strip()
                    line_split = [audio_code, text]
    #                 print(line_split)
                    text_df = text_df.append(pd.DataFrame(line_split, index=["filename", "text"]).transpose(), ignore_index=True)
            except:
                print("maybe incomplete line")


/home/user/shaheen.perveen/SER_new/IEMOCAP_text/Session1/
maybe incomplete line
maybe incomplete line
maybe incomplete line
maybe incomplete line
/home/user/shaheen.perveen/SER_new/IEMOCAP_text/Session2/
/home/user/shaheen.perveen/SER_new/IEMOCAP_text/Session3/
maybe incomplete line
maybe incomplete line
maybe incomplete line
maybe incomplete line
/home/user/shaheen.perveen/SER_new/IEMOCAP_text/Session4/
/home/user/shaheen.perveen/SER_new/IEMOCAP_text/Session5/


In [44]:
text_df.shape

(9937, 2)

In [45]:
len(text_df.filename.unique())

9801

In [46]:
text_df.head()

,filename,text
0,Ses01M_script03_2_M000,[LAUGHTER]
1,Ses01M_script03_2_F000,"[LAUGHTER] Utterly, utterly ridiculous."
2,Ses01M_script03_2_M001,We were very much younger then.
3,Ses01M_script03_2_F001,Very much sillier.
4,Ses01M_script03_2_M002,Mmhmm.


In [47]:
to_merge = labels_df[["wav_file", "emotion"]]
to_merge.columns = ["filename", "emotion"]

In [48]:
text_with_emotion = text_df.merge(to_merge, on='filename')

In [49]:
text_with_emotion.head()

,filename,text,emotion
0,Ses01M_script03_2_M000,[LAUGHTER],xxx
1,Ses01M_script03_2_F000,"[LAUGHTER] Utterly, utterly ridiculous.",xxx
2,Ses01M_script03_2_M001,We were very much younger then.,xxx
3,Ses01M_script03_2_F001,Very much sillier.,xxx
4,Ses01M_script03_2_M002,Mmhmm.,xxx


In [50]:
text_with_emotion.describe(include='all')

,filename,text,emotion
count,9753,9753,9753
unique,9753,7842,11
top,Ses05M_script02_2_M007,Yeah.,xxx
freq,1,81,2452


In [55]:
text_with_emotion.emotion.value_counts()

xxx    2452
fru    1806
neu    1629
ang    1079
exc    1028
sad    1022
hap     586
sur     106
fea      40
oth       3
dis       2
Name: emotion, dtype: int64

In [52]:
import unicodedata

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [53]:
text_with_emotion["processed_text"] = text_with_emotion["text"].apply(lambda x: normalizeString(x))

In [54]:
text_with_emotion.head()

,filename,text,emotion,processed_text
0,Ses01M_script03_2_M000,[LAUGHTER],xxx,laughter
1,Ses01M_script03_2_F000,"[LAUGHTER] Utterly, utterly ridiculous.",xxx,laughter utterly utterly ridiculous .
2,Ses01M_script03_2_M001,We were very much younger then.,xxx,we were very much younger then .
3,Ses01M_script03_2_F001,Very much sillier.,xxx,very much sillier .
4,Ses01M_script03_2_M002,Mmhmm.,xxx,mmhmm .


In [56]:
text_with_emotion.to_csv('/home/user/shaheen.perveen/SER_new/preprocess_info/preprocessed_transcription_text.csv', index=False)